<a href="https://colab.research.google.com/github/gayathritela/RCA-of-Cloud-Microservices/blob/main/Synthetic_datasets_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

### **Synthetic Datset Creation**

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import random
from datetime import datetime
from sklearn.ensemble import IsolationForest
import zipfile
from sentence_transformers import SentenceTransformer
import pinecone

def load_data(file_path):
    """ Load data from a CSV file. """
    return pd.read_csv(file_path)

def generate_synthetic_data(df, column_ranges, noise_level=0.1, tolerance_factor=0.3):
    """ Generate synthetic data with more variability. """
    synthetic_df = df.copy()
    for col in df.select_dtypes(include=[np.number]).columns:
        range_width = column_ranges[col]['max'] - column_ranges[col]['min']
        noise = np.random.normal(0, range_width * noise_level, size=len(df))
        synthetic_df[col] = df[col] + noise
        synthetic_df[col] = np.clip(synthetic_df[col], column_ranges[col]['min'], column_ranges[col]['max'])
    return synthetic_df

def dynamic_severity_adjustment(base_severity):
    """ Adjust severity based on the current time of day. """
    current_hour = datetime.now().hour
    adjusted_severity = base_severity * 1.2 if 8 <= current_hour <= 20 else base_severity * 0.8
    return adjusted_severity + random.uniform(-0.05, 0.05)

def inject_issue(df, path_set, column_ranges, issue_type='latency', base_severity=0.1):
    """ Inject an issue into both root cause node and target node and mark them accordingly in the issue_injected column.
        Additionally, include the path involved in each row. """
    suitable_columns = [col for col in df.columns if col.endswith('Average') or col.startswith('latency_')]
    issue_column = random.choice(suitable_columns)
    path = random.choice(path_set)
    microservice = random.choice(path)  # This will be considered as the root cause node.
    target_node = random.choice([node for node in path if node != microservice])  # Target node.

    severity = dynamic_severity_adjustment(base_severity)
    adjustment_factor = 1 + severity if issue_type == 'latency' else 1 - severity

    df.loc[:, issue_column] *= adjustment_factor
    df.loc[:, issue_column] = np.clip(df.loc[:, issue_column], column_ranges[issue_column]['min'], column_ranges[issue_column]['max'])

    df['issue_injected'] = df['microservice'].apply(
        lambda x: 'root_cause_node' if x == microservice else ('target_node' if x == target_node else 'no_issue'))

    path_str = ' -> '.join(path)
    df['path'] = path_str

    return df, {'target_node': target_node, 'root_cause_node': microservice, 'issue_type': issue_type, 'severity': severity, 'affected_column': issue_column, 'timestamp': str(datetime.now()), 'paths': path}

def save_data(df, ground_truth, output_dir, i):
    """ Save synthetic data and corresponding ground truth. """
    synthetic_file_path = os.path.join(output_dir, f'synthetic_data_{i}_issue.csv')
    ground_truth_file_path = os.path.join(output_dir, f'ground_truth_{i}_issue.json')
    df.to_csv(synthetic_file_path, index=False)
    with open(ground_truth_file_path, 'w') as f:
        json.dump(ground_truth, f)

def apply_isolation_forest(df, numeric_columns):
    """ Detect anomalies using the Isolation Forest algorithm. """
    isolation_forest = IsolationForest(contamination=0.2)
    isolation_forest.fit(df[numeric_columns])
    df['anomaly'] = isolation_forest.predict(df[numeric_columns])
    return df[df['anomaly'] == -1]

def evaluate_anomalies(df, ground_truth, numeric_columns):
    """ Evaluate detected anomalies against ground truth. """
    anomalies = apply_isolation_forest(df, numeric_columns)
    anomaly_indices = set(anomalies.index)
    injected_indices = set(df[df['issue_injected'] != 'no_issue'].index)
    correctly_detected = anomaly_indices.intersection(injected_indices)
    false_negatives = injected_indices.difference(anomaly_indices)
    false_positives = anomaly_indices.difference(injected_indices)

    evaluation_summary = {
        'correctly_detected': len(correctly_detected),
        'false_negatives': len(false_negatives),
        'false_positives': len(false_positives),
        'total_injected': len(injected_indices),
        'total_detected': len(anomaly_indices),
        'details': {
            'correctly_detected_indices': list(correctly_detected),
            'false_negative_indices': list(false_negatives),
            'false_positive_indices': list(false_positives)
        }
    }
    return evaluation_summary

def create_textual_representation(row):
    text = f"On {row['timestamp']}, the service {row['microservice']} reported metrics with "
    text += f"an availability of {row['availability_Average']}%, "
    text += f"an average latency of {row['latency_Average']} ms, "
    text += f"and peak latencies at p50: {row['latency_p50']} ms, p90: {row['latency_p90']} ms, p95: {row['latency_p95']} ms, "
    text += f"and p99: {row['latency_p99']} ms. Total requests were {row['requests_Sum']}. "

    if row['issue_injected'] == 'root_cause_node':
        text += f"This service was identified as the root cause of a performance issue. "
    elif row['issue_injected'] == 'target_node':
        text += f"This service was affected as a target node in the performance issue. "
    else:
        text += f"No significant issues were detected. "

    text += f"Service path involved: {row['path']}."

    return text


def main(path_set_file, original_file_paths, output_dir, num_datasets, column_ranges):
    """ Main function to process data and evaluate anomalies. """
    with open(path_set_file) as f:
        path_set = json.load(f)

    evaluations = []
    os.makedirs(output_dir, exist_ok=True)

    for i, file_path in enumerate(original_file_paths):
        df = load_data(file_path)
        for j in range(num_datasets):
            synthetic_df = generate_synthetic_data(df, column_ranges)
            synthetic_df, ground_truth = inject_issue(synthetic_df, path_set, column_ranges)

            synthetic_df['textual_representation'] = synthetic_df.apply(create_textual_representation, axis=1)

            save_data(synthetic_df, ground_truth, output_dir, f'{i}_{j}')

            numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
            evaluation = evaluate_anomalies(synthetic_df, ground_truth, numeric_columns)
            evaluations.append(evaluation)
            print(f"Evaluation for dataset {i}_{j}: {evaluation}")

    save_evaluation_results(evaluations, output_dir)
    zip_synthetic_data(output_dir, 'synthetic_data.zip')

def save_evaluation_results(evaluations, output_dir):
    """ Save a summary of the evaluations. """
    eval_df = pd.DataFrame(evaluations)
    eval_summary_path = os.path.join(output_dir, 'evaluations_summary.csv')
    eval_df.to_csv(eval_summary_path, index=False)
    print(eval_df)

def zip_synthetic_data(output_dir, zip_filename):
    """ Zip all synthetic data files into a single zip file. """
    with zipfile.ZipFile(os.path.join(output_dir, zip_filename), 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                if file.endswith('.csv') or file.endswith('.json'):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_dir))

if __name__ == "__main__":
    path_set_file = '/content/path_sets.json'
    original_file_paths = [
        '/content/transformed_transposed_issue0_metrics.csv',
        '/content/transformed_transposed_issue1_metrics.csv',
        '/content/transformed_transposed_issue2_metrics.csv'
    ]
    output_dir = '/content/Synthetic_datasets_v1'
    column_ranges = {
        'availability_Average': {'min': 0.0, 'max': 100.0},
        'latency_Average': {'min': 0.0, 'max': 4.91},
        'latency_p50': {'min': 0.0, 'max': 5.83},
        'latency_p90': {'min': 0.0, 'max': 6.60},
        'latency_p95': {'min': 0.0, 'max': 6.74},
        'latency_p99': {'min': 0.0, 'max': 6.76},
        'requests_Sum': {'min': 0.0, 'max': 1287.0}
    }
    main(path_set_file, original_file_paths, output_dir, 50, column_ranges)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 0_0: {'correctly_detected': 3, 'false_negatives': 7, 'false_positives': 39, 'total_injected': 10, 'total_detected': 42, 'details': {'correctly_detected_indices': [176, 178, 179], 'false_negative_indices': [165, 166, 167, 168, 169, 175, 177], 'false_positive_indices': [128, 129, 130, 131, 132, 5, 6, 7, 8, 9, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 37, 38, 171, 45, 46, 197, 203, 104, 119, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 0_1: {'correctly_detected': 5, 'false_negatives': 5, 'false_positives': 37, 'total_injected': 10, 'total_detected': 42, 'details': {'correctly_detected_indices': [5, 6, 7, 8, 9], 'false_negative_indices': [45, 46, 47, 48, 49], 'false_positive_indices': [128, 1, 129, 131, 130, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 156, 36, 171, 176, 179, 84, 86, 87, 88, 102, 119, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 2_47: {'correctly_detected': 1, 'false_negatives': 9, 'false_positives': 41, 'total_injected': 10, 'total_detected': 42, 'details': {'correctly_detected_indices': [48], 'false_negative_indices': [195, 196, 197, 198, 199, 45, 46, 47, 49], 'false_positive_indices': [128, 129, 131, 132, 5, 6, 7, 8, 9, 133, 134, 135, 136, 140, 141, 142, 143, 144, 150, 152, 155, 35, 36, 37, 171, 175, 176, 80, 81, 82, 83, 84, 103, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 2_48: {'correctly_detected': 8, 'false_negatives': 2, 'false_positives': 34, 'total_injected': 10, 'total_detected': 42, 'details': {'correctly_detected_indices': [135, 136, 137, 138, 139, 140, 142, 144], 'false_negative_indices': [141, 143], 'false_positive_indices': [128, 129, 130, 131, 132, 5, 6, 7, 8, 9, 133, 134, 150, 36, 38, 39, 175, 48, 176, 80, 81, 82, 83, 84, 110, 113, 120, 121, 122, 123, 124, 125, 126, 127]}}
Evaluation for dataset 2_49: {'correctly_detected': 0, 'false_negatives': 10, 'false_positives': 42, 'total_injected': 10, 'total_detected': 42, 'details': {'correctly_detected_indices': [], 'false_negative_indices': [0, 1, 2, 3, 4, 180, 181, 182, 183, 184], 'false_positive_indices': [128, 129, 130, 132, 5, 6, 7, 8, 9, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 150, 37, 38, 39, 46, 175, 49, 80, 81, 82, 83, 84, 113, 120, 121, 122, 123, 124, 125, 126, 127]}}
     correctly_detected  false_negatives  false_positives  total_injected  \

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


In [ ]:
test

In [ ]:
import pandas as pd
import numpy as np
import json
import os
import random
from datetime import datetime
from sklearn.ensemble import IsolationForest
import zipfile
from sentence_transformers import SentenceTransformer

def load_data(file_path):
    """ Load data from a CSV file. """
    return pd.read_csv(file_path)

def generate_synthetic_data(df, column_ranges, noise_level=0.1, tolerance_factor=0.3):
    """ Generate synthetic data with more variability. """
    synthetic_df = df.copy()
    for col in df.select_dtypes(include=[np.number]).columns:
        range_width = column_ranges[col]['max'] - column_ranges[col]['min']
        noise = np.random.normal(0, range_width * noise_level, size=len(df))
        synthetic_df[col] = df[col] + noise
        synthetic_df[col] = np.clip(synthetic_df[col], column_ranges[col]['min'], column_ranges[col]['max'])
    return synthetic_df

def dynamic_severity_adjustment(base_severity):
    """ Adjust severity based on the current time of day. """
    current_hour = datetime.now().hour
    adjusted_severity = base_severity * 1.2 if 8 <= current_hour <= 20 else base_severity * 0.8
    return adjusted_severity + random.uniform(-0.05, 0.05)

def inject_issue(df, path_set, column_ranges, issue_type='latency', base_severity=0.1):
    """ Inject an issue into both root cause node and target node and mark them accordingly in the issue_injected column.
        Additionally, include the path involved in each row. """
    suitable_columns = [col for col in df.columns if col.endswith('Average') or col.startswith('latency_')]
    issue_column = random.choice(suitable_columns)
    path = random.choice(path_set)
    microservice = random.choice(path)  # This will be considered as the root cause node.
    target_node = random.choice([node for node in path if node != microservice])  # Target node.

    severity = dynamic_severity_adjustment(base_severity)
    adjustment_factor = 1 + severity if issue_type == 'latency' else 1 - severity

    df.loc[:, issue_column] *= adjustment_factor
    df.loc[:, issue_column] = np.clip(df.loc[:, issue_column], column_ranges[issue_column]['min'], column_ranges[issue_column]['max'])

    df['issue_injected'] = df['microservice'].apply(
        lambda x: 'root_cause_node' if x == microservice else ('target_node' if x == target_node else 'no_issue'))

    path_str = ' -> '.join(path)
    df['path'] = path_str

    return df, {'target_node': target_node, 'root_cause_node': microservice, 'issue_type': issue_type, 'severity': severity, 'affected_column': issue_column, 'timestamp': str(datetime.now()), 'paths': path}

def create_textual_representation(row):
    text = f"On {row['timestamp']}, the service {row['microservice']} reported metrics with "
    text += f"an availability of {row['availability_Average']}%, "
    text += f"an average latency of {row['latency_Average']} ms, "
    text += f"and peak latencies at p50: {row['latency_p50']} ms, p90: {row['latency_p90']} ms, p95: {row['latency_p95']} ms, "
    text += f"and p99: {row['latency_p99']} ms. Total requests were {row['requests_Sum']}. "

    if row['issue_injected'] == 'root_cause_node':
        text += f"This service was identified as the root cause of a performance issue. "
    elif row['issue_injected'] == 'target_node':
        text += f"This service was affected as a target node in the performance issue. "
    else:
        text += f"No significant issues were detected. "

    text += f"Service path involved: {row['path']}."
    return text

def save_data(df, ground_truth, output_dir, i):
    """ Save synthetic data and corresponding ground truth. """
    synthetic_file_path = os.path.join(output_dir, f'synthetic_data_{i}_issue.csv')
    ground_truth_file_path = os.path.join(output_dir, f'ground_truth_{i}_issue.json')
    df.to_csv(synthetic_file_path, index=False)
    with open(ground_truth_file_path, 'w') as f:
        json.dump(ground_truth, f)

def apply_isolation_forest(df, numeric_columns):
    """ Detect anomalies using the Isolation Forest algorithm. """
    isolation_forest = IsolationForest(contamination=0.2)
    isolation_forest.fit(df[numeric_columns])
    df['anomaly'] = isolation_forest.predict(df[numeric_columns])
    return df[df['anomaly'] == -1]

def evaluate_anomalies(df, ground_truth, numeric_columns):
    """ Evaluate detected anomalies against ground truth. """
    anomalies = apply_isolation_forest(df, numeric_columns)
    anomaly_indices = set(anomalies.index)
    injected_indices = set(df[df['issue_injected'] != 'no_issue'].index)
    correctly_detected = anomaly_indices.intersection(injected_indices)
    false_negatives = injected_indices.difference(anomaly_indices)
    false_positives = anomaly_indices.difference(injected_indices)

    evaluation_summary = {
        'correctly_detected': len(correctly_detected),
        'false_negatives': len(false_negatives),
        'false_positives': len(false_positives),
        'total_injected': len(injected_indices),
        'total_detected': len(anomaly_indices),
        'details': {
            'correctly_detected_indices': list(correctly_detected),
            'false_negative_indices': list(false_negatives),
            'false_positive_indices': list(false_positives)
        }
    }
    return evaluation_summary

def main(path_set_file, original_file_paths, output_dir, num_datasets, column_ranges):
    """ Main function to process data and evaluate anomalies. """
    with open(path_set_file) as f:
        path_set = json.load(f)

    evaluations = []
    os.makedirs(output_dir, exist_ok=True)

    for i, file_path in enumerate(original_file_paths):
        df = load_data(file_path)
        for j in range(num_datasets):
            synthetic_df = generate_synthetic_data(df, column_ranges)
            synthetic_df, ground_truth = inject_issue(synthetic_df, path_set, column_ranges)

            synthetic_df['textual_representation'] = synthetic_df.apply(create_textual_representation, axis=1)

            save_data(synthetic_df, ground_truth, output_dir, f'{i}_{j}')

            numeric_columns = df.select_dtypes(include=[np.number]).columns.tolist()
            evaluation = evaluate_anomalies(synthetic_df, ground_truth, numeric_columns)
            evaluations.append(evaluation)
            print(f"Evaluation for dataset {i}_{j}: {evaluation}")

    save_evaluation_results(evaluations, output_dir)
    zip_synthetic_data(output_dir, 'synthetic_data.zip')

def save_evaluation_results(evaluations, output_dir):
    """ Save a summary of the evaluations. """
    eval_df = pd.DataFrame(evaluations)
    eval_summary_path = os.path.join(output_dir, 'evaluations_summary.csv')
    eval_df.to_csv(eval_summary_path, index=False)
    print(eval_df)

def zip_synthetic_data(output_dir, zip_filename):
    """ Zip all synthetic data files into a single zip file. """
    with zipfile.ZipFile(os.path.join(output_dir, zip_filename), 'w') as zipf:
        for root, _, files in os.walk(output_dir):
            for file in files:
                if file.endswith('.csv') or file.endswith('.json'):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), output_dir))

if __name__ == "__main__":
    path_set_file = '/content/path_sets.json'
    original_file_paths = [
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue17_metrics.csv',
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue13_metrics.csv',
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue10_metrics.csv',
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue2_metrics.csv',
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue6_metrics.csv',
        '/content/drive/MyDrive/DATA/Transformed_metric_high_test/transformed_transposed_issue3_metrics.csv',


    ]
    output_dir = '/content/High Traffic_test_Synthetic_datasets'
    column_ranges = {
        'availability_Average': {'min': 0.0, 'max': 100.0},
        'latency_Average': {'min': 0.0, 'max': 4.91},
        'latency_p50': {'min': 0.0, 'max': 5.83},
        'latency_p90': {'min': 0.0, 'max': 6.60},
        'latency_p95': {'min': 0.0, 'max': 6.74},
        'latency_p99': {'min': 0.0, 'max': 6.76},
        'requests_Sum': {'min': 0.0, 'max': 1287.0}
    }
    main(path_set_file, original_file_paths, output_dir, 10, column_ranges)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 0_0: {'correctly_detected': 0, 'false_negatives': 10, 'false_positives': 44, 'total_injected': 10, 'total_detected': 44, 'details': {'correctly_detected_indices': [], 'false_negative_indices': [65, 66, 67, 68, 69, 185, 186, 187, 188, 189], 'false_positive_indices': [128, 129, 130, 132, 5, 6, 7, 8, 9, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 152, 35, 36, 37, 38, 45, 47, 175, 180, 181, 82, 83, 84, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 0_1: {'correctly_detected': 3, 'false_negatives': 7, 'false_positives': 41, 'total_injected': 10, 'total_detected': 44, 'details': {'correctly_detected_indices': [36, 37, 39], 'false_negative_indices': [35, 38, 185, 186, 187, 188, 189], 'false_positive_indices': [128, 129, 130, 131, 5, 6, 7, 8, 9, 133, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 152, 157, 46, 180, 181, 182, 184, 82, 83, 84, 115, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


...

Evaluation for dataset 5_7: {'correctly_detected': 4, 'false_negatives': 6, 'false_positives': 40, 'total_injected': 10, 'total_detected': 44, 'details': {'correctly_detected_indices': [35, 37, 38, 39], 'false_negative_indices': [0, 1, 2, 3, 4, 36], 'false_positive_indices': [128, 129, 131, 5, 6, 7, 8, 9, 134, 135, 138, 141, 143, 182, 183, 184, 187, 189, 68, 197, 199, 80, 81, 82, 83, 84, 212, 213, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 5_8: {'correctly_detected': 3, 'false_negatives': 7, 'false_positives': 41, 'total_injected': 10, 'total_detected': 44, 'details': {'correctly_detected_indices': [187, 37, 38], 'false_negative_indices': [35, 36, 39, 185, 186, 188, 189], 'false_positive_indices': [128, 129, 132, 5, 6, 7, 8, 9, 138, 142, 163, 45, 48, 49, 180, 181, 182, 183, 184, 68, 197, 206, 80, 81, 82, 83, 84, 212, 213, 115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127]}}


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


Evaluation for dataset 5_9: {'correctly_detected': 5, 'false_negatives': 0, 'false_positives': 39, 'total_injected': 5, 'total_detected': 44, 'details': {'correctly_detected_indices': [128, 129, 125, 126, 127], 'false_negative_indices': [], 'false_positive_indices': [131, 5, 6, 7, 8, 9, 136, 138, 143, 144, 35, 36, 37, 38, 39, 42, 47, 180, 181, 182, 183, 184, 187, 68, 197, 199, 80, 81, 82, 83, 84, 212, 116, 117, 120, 121, 122, 123, 124]}}
    correctly_detected  false_negatives  false_positives  total_injected  \
0                    0               10               44              10   
1                    3                7               41              10   
2                    0               10               44              10   
3                    0               10               44              10   
4                    1                9               43              10   
5                    0               10               44              10   
6                   10    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/Synthetic_datasets_v1

/content/Synthetic_datasets_v1


In [ ]:
!zip -r Synthetic_dataset.zip /content/Synthetic_datasets_v1

  adding: content/Synthetic_datasets_v1/ (stored 0%)
  adding: content/Synthetic_datasets_v1/synthetic_data_0_20_issue.csv (deflated 89%)
  adding: content/Synthetic_datasets_v1/ground_truth_0_8_issue.json (deflated 50%)
  adding: content/Synthetic_datasets_v1/ground_truth_2_28_issue.json (deflated 51%)
  adding: content/Synthetic_datasets_v1/synthetic_data_0_46_issue.csv (deflated 89%)
  adding: content/Synthetic_datasets_v1/ground_truth_1_26_issue.json (deflated 37%)
  adding: content/Synthetic_datasets_v1/synthetic_data_1_9_issue.csv (deflated 89%)
  adding: content/Synthetic_datasets_v1/synthetic_data_1_47_issue.csv (deflated 87%)
  adding: content/Synthetic_datasets_v1/ground_truth_0_39_issue.json (deflated 36%)
  adding: content/Synthetic_datasets_v1/synthetic_data_2_11_issue.csv (deflated 90%)
  adding: content/Synthetic_datasets_v1/synthetic_data_1_35_issue.csv (deflated 88%)
  adding: content/Synthetic_datasets_v1/synthetic_data_2_13_issue.csv (deflated 88%)
  adding: content/

In [ ]:
from google.colab import files
files.download('dataset.zip')


In [ ]:
import pandas as pd
import os

directory = '/content/Synthetic_datasets'
csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]

combined_datasets_df = pd.DataFrame()

for file_name in csv_files:
    file_path = os.path.join(directory, file_name)
    df = pd.read_csv(file_path)
    df['source_file'] = file_name
    combined_datasets_df = pd.concat([combined_datasets_df, df], ignore_index=True)


In [ ]:
import json

json_files = [f for f in os.listdir(directory) if f.endswith('.json')]
ground_truth_data = []

for file_name in json_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        data['source_file'] = file_name
        ground_truth_data.append(data)

ground_truth_df = pd.DataFrame(ground_truth_data)


In [ ]:
combined_datasets_df.to_csv('/content/combined_Synthetic_datasets.csv', index=False)
ground_truth_df.to_csv('/content/combined_Synthetic_ground_truth.csv', index=False)
